In [ ]:
import numpy
import matplotlib
import lxml
import pandas
import pyteomics
import csv
import math
import multiprocessing
from pyteomics import mass

In [ ]:
pwd

In [ ]:
vclh_seq = 'MNHKVHMHHHHHHADEQEEKAKVRTELIQELAQGLGGIEKKNFPTLGDEDLDHTYMTKLLTYLQEREQAENSWRKRLLKGIQDHALDLVPRGSPGLPGPRGEQGPTGPTGPAGPRGLQGLQGLQGERGEQGPTGPAGPRGLQGERGEQGPTGLAGKAGEAGAKGETGPAGPQGPRGEQGPQGLPGKDGEAGAQGRPGKRGKQGQKGEKGEPGTQGAKGDRGETGPVGPRGERGEAGPAGKDGERGPVGPAGKDGQNGQDGLPGKDGKDGQNGKDGLPGKDGKDGQNGKDGLPGKDGKDGQDGKDGLPGKDGKDGLPGKDGKDGQPGKPGKY'

In [ ]:
num_seq = 'MNHKVHMHHHHHHADEQEEKAKVRTELIQELAQGLGGIEKKNFPTLGDEDLDHTYMTKLLTYL8QE7REQAENS6W5RKRLLKG4IQD3HA2L1DLVPRGSPGLPGPRGEQGPTGPTGPAGPRGLQGLQGLQGERGEQGPTGPAGPRGLQGERGEQGPTGLAGKAGEAGAKGETGPAGPQGPRGEQGPQGLPGKDGEAGAQGRPGKRGKQGQKGEKGEPGTQGAKGDRGETGPVGPRGERGEAGPAGKDGERGPVGPAGKDGQNGQDGLPGKDGKDGQNGKDGLPGKDGKDGQNGKDGLPGKDGKDGQDGKDGLPGKDGKDGLPGKDGKDGQPGKPGKY'

In [ ]:
num_seq_list = [num_seq]

In [ ]:
writer = pandas.ExcelWriter('try.xlsx', engine='xlsxwriter')
workbook = writer.book
red = workbook.add_format({'color': 'red'})

In [ ]:
index = ['1', '2', '3', '4', '5', '6', '7', '8', '9', '10', '11', '12', '13', '14', '15', '16']

In [ ]:
format_seq = []
for base in num_seq:
    if base in index:
    #if base == 'M':
        format_seq.extend((red, base))
    else:
        format_seq.append(base)
print(format_seq)

In [ ]:
play1 = '/Users/user/Documents/fragment_finder/files/VCLH_T-145-DSP-04_input.csv'

In [ ]:
play1[:-3]

In [ ]:
amino_list[0] = amino_list[0] + '1'

In [ ]:
rejoined = ''
print(rejoined.join(amino_list))

In [ ]:
type(vclh_seq)

In [ ]:
file = 'C:\\Users\\ray07c\\Documents\\Parkville_data\\fragment_finder\\files\\VCLH_T-145-DSP-04_input.csv'

In [ ]:
def mass_cal(peptide_seq):
    return(round(mass.calculate_mass(peptide_seq, average = True), 1))

In [ ]:
mass_cal(vclh_seq)

In [ ]:
def import_dataframe(file_location):
    data = pandas.read_csv(file_location)
    data.rename(columns={'m/z':'M(obs)'}, inplace=True)
    return(data)

In [ ]:
def import_obs_masses(dataframe):
    return(list(dataframe['M(obs)']))

In [ ]:
%load_ext lprun

In [ ]:
def fragments2(prot_seq, obs_masses, dataframe, tolerance):

    single_cut = []
    double_cut = []
    start = 0
    s = int(min(obs_masses)//105)
    e = int(max(obs_masses)//90)
    for frag in prot_seq:
        for i in range(s, e):
            if i > len(prot_seq):
                break
            for num in obs_masses:
                if math.isclose(round(mass.calculate_mass(prot_seq[start:i], average = True), 1), num, abs_tol = tolerance):
                    if i == len(prot_seq):
                        find = [prot_seq[start] + str(start + 1),
                                str(i),
                                num, 
                                round(mass.calculate_mass(prot_seq[start:i], average = True), 1),
                                round(num - round(mass.calculate_mass(prot_seq[start:i], average = True), 1), 1)]
                        single_cut.append(find)
                    else:
                        find = [prot_seq[start] + str(start + 1),
                                str(i),
                                num, 
                                round(mass.calculate_mass(prot_seq[start:i], average = True), 1),
                                round(num - round(mass.calculate_mass(prot_seq[start:i], average = True), 1), 1)]
                        double_cut.append(find)
        s += 1
        e += 1
        start += 1
    
    df1 = pandas.DataFrame(single_cut, columns = ['Cutsite (Nterm)', 'Cterm', 'M(obs)', 'M(calc)', 'deltaM'])
    df1.sort_values('M(obs)', inplace=True)
    df2 = pandas.DataFrame(double_cut, columns = ['Cutsite (Nterm)', 'Cutsite (Cterm)', 'M(obs)', 'M(calc)', 'deltaM'])
    df2.sort_values('M(obs)', inplace=True)
    df_i = dataframe[['M(obs)', 'I']]
    df1_i = pandas.merge(df1, df_i, on= 'M(obs)', how='right')
    df1_i.dropna(how = 'any', inplace = True)
    percent_i = [round(((num / max(df1_i['I'])) * 100), 2) for num in df1_i['I']]
    df1_i['I'] = percent_i
    df1_i.rename(columns={'I':'% Intensity'}, inplace=True)

    print(df1_i.to_string(index=False))
    print(df2.to_string(index=False))

In [ ]:
dataframe = import_dataframe(file)
whole_prot_mass = mass_cal(vclh_seq)
observed_masses = import_obs_masses(dataframe)

In [ ]:
%%time
fragments2(vclh_seq, observed_masses, dataframe, 0.5)

In [ ]:
def fragments_multi(prot_seq, obs_mass, tolerance):

    found = []
    start = 0
    s = int(obs_mass)//107
    e = int(obs_mass)//96
    for frag in prot_seq:
        for i in range(s, e):
            if math.isclose(round(mass.calculate_mass(prot_seq[start:i], average = True), 1), obs_mass, abs_tol = tolerance):
                if prot_seq[start:i] not in found:
                    found.append(prot_seq[start:i]) 
                    found.append(round(mass.calculate_mass(prot_seq[start:i], average = True), 1))
        s += 1
        e += 1
        start += 1
    if len(found) != 0:
        print(found)

In [ ]:
multi = [(vclh_seq, mass, 0.5) for mass in observed_masses]

In [ ]:
list = ['A', 'B', '2C']

In [ ]:
print(str(list))

In [ ]:
%%time
if __name__ == '__main__':
    with multiprocessing.Pool(processes=2) as pool:
        results = pool.starmap(fragments_multi, multi)
    print(results)

In [ ]:
final = [[['double', 'Y62', '300', 23621.5, 23621.3, 0.2], ['single', 'D87', '331', 23621.5, 23621.2, 0.3]],
 [['single', 'L86', '331', 23734.2, 23734.4, -0.2]],
 [['double', 'N2', '231', 23942.9, 23943.1, -0.2], ['double', 'L51', '290', 23942.9, 23942.8, 0.1], ['double', 'D52', '291', 23942.9, 23942.8, 0.1], ['single', 'H84', '331', 23942.9, 23942.6, 0.3]],
 [['double', 'H3', '236', 24299.4, 24299.5, -0.1], ['single', 'I81', '331', 24299.4, 24299.0, 0.4]],
 [['double', 'L51', '294', 24338.6, 24338.3, 0.3], ['double', 'E70', '318', 24338.6, 24339.1, -0.5]],
 [['single', 'R74', '331', 25150.9, 25151.1, -0.2]],
 [['single', 'W73', '331', 25337.1, 25337.3, -0.2]],
 [],
 [['single', 'R66', '331', 26152.1, 26152.1, 0.0]],
 [['double', 'K4', '260', 26409.1, 26408.6, 0.5], ['single', 'Q64', '331', 26409.1, 26409.3, -0.2]]]


In [ ]:
combined1 = [index for line in final for index in line]
#print(combined1)

In [ ]:
df_final_final = pandas.DataFrame(combined1 , columns = ['# Cuts', 'Cutsite (Nterm)', 'Cterm', 'M(obs)', 'M(calc)', 'deltaM'])
df_final_final.sort_values(['# Cuts', 'M(obs)'], ascending = [False, True], inplace=True)
df_final_final['I'] = [100.0, 58.93, 70.04, 79.22, 16.45, 36.61, 75.65, 11.83, 100.0, 70.04, 70.04, 70.04, 79.22, 6.03, 6.03, 11.83]
df_final_final.style.background_gradient(cmap='viridis', low = min(df_final_final['I'])/100, high = 0.5, subset = 'I')
df_final_final['I']
#(cmap='PuBu', low=0, high=0, axis=0, subset=None)
#df_final_final.sort_values('M(obs)', inplace=True)
#df_final_final

In [ ]:
def fragments_speedup(prot_seq, obs_mass, mass_diffs, tolerance):

    found = []
    start = 0
    s = mass_diffs
    e = len(prot_seq)
    for frag in prot_seq:
        for i in range(s, e):
            if math.isclose(round(mass.calculate_mass(prot_seq[start:i], average = True), 1), obs_mass, abs_tol = tolerance):
                if prot_seq[start:i] not in found:
                    found.append(prot_seq[start:i]) 
                    found.append(round(mass.calculate_mass(prot_seq[start:i], average = True), 1))
        s += 1
        e += 1
        start += 1
    print(found)
    
pool = multiprocessing.Pool(processes=2)
whole_prot_mass = mass_cal(vclh_seq)
observed_masses = import_obs_masses(file)
mass_differences = mass_diff(whole_prot_mass, observed_masses)

input_list = []

part_list = []
for mass in observed_masses[0:2]:
    part_list = (vclh_seq, mass, mass_differences, 0.5)
    input_list.append(part_list)
#print(input_list)

%%time
pool.starmap(fragments_speedup, input_list)
pool.close()